In [ ]:
import nltk
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
# Download stop words list and punkt tokenizer (if not previously downloaded)
nltk.download('stopwords')
nltk.download('punkt')
!pip install gensim
!pip install pyLDAvis
!pip install wordcloud matplotlib
!pip install emoji

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
import re
import nltk
from nltk.corpus import stopwords
from collections import defaultdict, Counter
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
!wget -P data/full_dataset/ https://storage.googleapis.com/gresearch/goemotions/data/full_dataset/goemotions_1.csv
!wget -P data/full_dataset/ https://storage.googleapis.com/gresearch/goemotions/data/full_dataset/goemotions_2.csv
!wget -P data/full_dataset/ https://storage.googleapis.com/gresearch/goemotions/data/full_dataset/goemotions_3.csv

--2025-02-24 10:52:58--  https://storage.googleapis.com/gresearch/goemotions/data/full_dataset/goemotions_1.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.31.207, 142.251.111.207, 142.251.179.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.31.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14174600 (14M) [application/octet-stream]
Saving to: ‘data/full_dataset/goemotions_1.csv.2’

goemotions_1.csv.2  100%[===================>]  13.52M  --.-KB/s    in 0.1s    

2025-02-24 10:52:59 (122 MB/s) - ‘data/full_dataset/goemotions_1.csv.2’ saved [14174600/14174600]

--2025-02-24 10:52:59--  https://storage.googleapis.com/gresearch/goemotions/data/full_dataset/goemotions_2.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.31.207, 142.251.111.207, 142.251.179.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.31.207|:443... connected.
HTTP request sent, awaitin

In [ ]:
goemotions_1 = pd.read_csv('data/full_dataset/goemotions_1.csv')
goemotions_2 = pd.read_csv('data/full_dataset/goemotions_2.csv')
goemotions_3 = pd.read_csv('data/full_dataset/goemotions_3.csv')

combined_df = pd.concat([goemotions_1, goemotions_2, goemotions_3], ignore_index=True)
emotion_columns = combined_df.columns[9:]

combined_df['created_utc'] = pd.to_datetime(combined_df['created_utc'], unit='s', errors='coerce')
combined_df.to_csv('data/full_dataset/goemotions_combined.csv', index=False)

#emoji

In [ ]:
import pandas as pd
import re
import emoji

# Dictionary to store emojis with their positions, texts, and associated labels (emotions)
emoji_dict = {}

# Function to extract emojis and build the dictionary with emotions
def build_emoji_dict(df):
    # Iterate through each row of the DataFrame
    for idx, row in df.iterrows():
        text = row['text']

        # Iterate through the emotion columns
        for emotion_column in emotion_columns:
            if row[emotion_column] == 1:  # Check if the emotion is present
                emotion = emotion_column  # Get the emotion from the column name

                # Iterate through each character in the text to check for emojis
                for char in text:
                    if emoji.is_emoji(char):
                        if char not in emoji_dict:
                            emoji_dict[char] = {'texts': [], 'emotion_count': {}}
                        emoji_dict[char]['texts'].append((idx, text))

                        # Update the emotion count for the current emoji
                        if emotion not in emoji_dict[char]['emotion_count']:
                            emoji_dict[char]['emotion_count'][emotion] = 0
                        emoji_dict[char]['emotion_count'][emotion] += 1

build_emoji_dict(combined_df)
emoji_summary = []

# Populate the summary DataFrame with emoji and their associated emotions
for emoji_char, emotions in emoji_dict.items():
    row = {'Emoji': emoji_char}
    row.update(emotions)  # Add emotion counts to the row
    emoji_summary.append(row)

# Create the DataFrame from the list of rows
emoji_df = pd.DataFrame(emoji_summary)

# Display the resulting table
print(emoji_df)

    Emoji                                              texts  \
0       😇  [(102, Nice! You and her would get along i bet...   
1       🤔  [(132, Funny, the right only seems to say this...   
2       😍  [(168, I love this sub reddit 😍😂), (416, Looks...   
3       😂  [(168, I love this sub reddit 😍😂), (340, I'm s...   
4       🤣  [(246, That’s what’s it’s called or what SHE’S...   
..    ...                                                ...   
257     🐢  [(146943, It took me 14 years to secure my AA ...   
258     🚀  [(148977, Haha, thanks, bot! How's your day go...   
259     🍧  [(151078, How are you all losing weight post-s...   
260     💥  [(156936, 😂😂👸👸👏🙌👏🙌❤❤❤💥💥👑👑💎💎💯💯♀️♀️♀️ Now that's...   
261     🍑  [(158002, Highkey tho! He got ass! 🍑), (200908...   

                                         emotion_count  
0    {'neutral': 3, 'realization': 1, 'surprise': 1...  
1    {'amusement': 9, 'confusion': 17, 'approval': ...  
2    {'love': 29, 'admiration': 35, 'neutral': 20, ...  
3  

In [ ]:
emoji_dict['🤭']

{'texts': [(39569,
   'But then you missed their officiant who looks and sounds like [NAME]! 🤭😳🤣 '),
  (128263,
   'But then you missed their officiant who looks and sounds like [NAME]! 🤭😳🤣 '),
  (147449,
   'But then you missed their officiant who looks and sounds like [NAME]! 🤭😳🤣 ')],
 'emotion_count': {'neutral': 3}}

In [ ]:
emoji_df[['Emoji','emotion_count']]

,Emoji,emotion_count
0,😇,"{'neutral': 3, 'realization': 1, 'surprise': 1..."
1,🤔,"{'amusement': 9, 'confusion': 17, 'approval': ..."
2,😍,"{'love': 29, 'admiration': 35, 'neutral': 20, ..."
3,😂,"{'love': 76, 'disappointment': 38, 'approval':..."
4,🤣,"{'curiosity': 17, 'disappointment': 11, 'appro..."
...,...,...
257,🐢,"{'neutral': 1, 'realization': 2}"
258,🚀,"{'amusement': 1, 'curiosity': 3, 'gratitude': 3}"
259,🍧,"{'curiosity': 2, 'realization': 1, 'annoyance'..."
260,💥,"{'approval': 4, 'love': 2, 'excitement': 2, 'n..."


In [ ]:
def normalize_emotions(emotion_count):
    total_count = sum(emotion_count.values())
    return {emotion: count / total_count for emotion, count in emotion_count.items()}


emoji_df['normalized_emotion_count'] = emoji_df['emotion_count'].apply(normalize_emotions)

emotion_columns = set()
for emotion_count in emoji_df['normalized_emotion_count']:
    emotion_columns.update(emotion_count.keys())

# צור את המטריצה
emotion_matrix = pd.DataFrame(columns=sorted(emotion_columns))

# מלא את המטריצה עבור כל אימוג'י
for idx, row in emoji_df.iterrows():
    emoji = row['Emoji']
    normalized_counts = row['normalized_emotion_count']
    emotion_matrix.loc[emoji] = [normalized_counts.get(emotion, 0) for emotion in emotion_matrix.columns]

# הצג את המטריצה
print(emotion_matrix)



    admiration  amusement     anger  annoyance  approval    caring  confusion  \
😇     0.111111   0.055556  0.000000   0.000000  0.055556  0.000000   0.055556   
🤔     0.022222   0.050000  0.033333   0.066667  0.072222  0.005556   0.094444   
😍     0.198864   0.011364  0.000000   0.000000  0.045455  0.011364   0.000000   
😂     0.044614   0.175271  0.015296   0.048438  0.112173  0.010198   0.019120   
🤣     0.126984   0.176871  0.029478   0.027211  0.063492  0.011338   0.004535   
..         ...        ...       ...        ...       ...       ...        ...   
🐢     0.000000   0.000000  0.000000   0.000000  0.000000  0.000000   0.000000   
🚀     0.000000   0.142857  0.000000   0.000000  0.000000  0.000000   0.000000   
🍧     0.000000   0.000000  0.000000   0.250000  0.000000  0.000000   0.000000   
💥     0.000000   0.000000  0.000000   0.000000  0.400000  0.000000   0.000000   
🍑     0.000000   0.000000  0.000000   0.000000  0.500000  0.000000   0.000000   

    curiosity    desire  di

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd


emotion_columns = list(set([key for emo_dict in emoji_df['emotion_count'] for key in emo_dict.keys()]))
emotion_matrix = pd.DataFrame(0, index=emoji_df['Emoji'], columns=emotion_columns)

for idx, row in emoji_df.iterrows():
    emoji = row['Emoji']
    emotion_counts = row['emotion_count']
    total_count = sum(emotion_counts.values())
    for emotion, count in emotion_counts.items():
        emotion_matrix.loc[emoji, emotion] = count / total_count

cosine_sim = cosine_similarity(emotion_matrix)

cosine_sim_df = pd.DataFrame(cosine_sim, index=emotion_matrix.index, columns=emotion_matrix.index)

plt.figure(figsize=(500, 400))
sns.heatmap(cosine_sim_df, annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5)

plt.title("Emoji Similarity Based on Emotional Distribution", fontsize=100)
plt.xlabel("Emojis", fontsize=88)
plt.ylabel("Emojis", fontsize=88)

plt.show()


<ipython-input-7-fee29c2b2089>:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.16666666666666666' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  emotion_matrix.loc[emoji, emotion] = count / total_count
<ipython-input-7-fee29c2b2089>:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.05555555555555555' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  emotion_matrix.loc[emoji, emotion] = count / total_count
<ipython-input-7-fee29c2b2089>:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.05555555555555555' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  emotion_matrix.loc[emoji, emotion] = count / total_count
<ipytho

#preprocess

In [ ]:
def preprocess_text(text, stop_words):
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'_', ' ', text)
    words = text.lower().split()
    words = [word for word in words if word not in stop_words]
    return words

negation_words = ["not", "no", "never", "none", "nobody", "nothing", "neither", "nor", "nowhere", "isn't", "aren't", "wasn't", "weren't", "don't", "doesn't", "didn't", "won't", "wouldn't", "shan't", "shouldn't", "can't", "cannot", "couldn't", "mightn't", "mustn't","don't"]
stop_words = set(stopwords.words('english'))

custom_stop_words = [word for word in stop_words if word not in negation_words]

combined_df['preprocessed_text'] = combined_df['text'].apply(lambda x: preprocess_text(x, custom_stop_words))

In [ ]:
combined_df[['text','preprocessed_text']]

,text,preprocessed_text
0,That game hurt.,"[game, hurt]"
1,>sexuality shouldn’t be a grouping category I...,"[sexuality, grouping, category, makes, differe..."
2,"You do right, if you don't care then fuck 'em!","[right, care, fuck, em]"
3,Man I love reddit.,"[man, love, reddit]"
4,"[NAME] was nowhere near them, he was by the Fa...","[name, nowhere, near, falcon]"
...,...,...
211220,Everyone likes [NAME].,"[everyone, likes, name]"
211221,Well when you’ve imported about a gazillion of...,"[well, imported, gazillion, country, gets, ser..."
211222,That looks amazing,"[looks, amazing]"
211223,The FDA has plenty to criticize. But like here...,"[fda, plenty, criticize, like, usually, critic..."


In [ ]:
from collections import defaultdict

def build_emotion_word_dict(dataframe, text_column, emotion_columns):
    """
    Builds a dictionary where each key is an emotion, and the value is a list of unique words
    that appeared in texts associated with that emotion.

    :param dataframe: DataFrame containing a text column and emotion columns.
    :param text_column: The name of the column containing the text data.
    :param emotion_columns: A list of columns representing emotions.
    :return: A dictionary mapping emotions to lists of words.
    """
    # Initialize a defaultdict with lists
    emotion_word_dict = defaultdict(list)

    # Iterate over each row in the DataFrame
    for _, row in dataframe.iterrows():
        # Extract the words from the text column
        text_words = row[text_column]
        for emotion in emotion_columns:
            if row[emotion] == 1:  # Check if the emotion is associated with this text
                emotion_word_dict[emotion].extend(text_words)

    # Remove duplicate words for each emotion
    # for emotion, words in emotion_word_dict.items():
    #     emotion_word_dict[emotion] = list(set(words))

    return emotion_word_dict

# Example usage
emotion_word_dict = build_emotion_word_dict(combined_df, text_column='preprocessed_text', emotion_columns=emotion_columns)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import re
from collections import Counter
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS, TfidfVectorizer

def get_common_words(subreddit_texts, stop_words=set(), top_n=10):
    all_text = " ".join(subreddit_texts)
    words = re.findall(r'\b[a-zA-Z]+\b', all_text.lower())
    words = [w for w in words if w not in stop_words]
    return Counter(words).most_common(top_n)

def get_ngrams(subreddit_texts, n=2, stop_words=set(), top_n=10):
    all_text = " ".join(subreddit_texts)
    words = re.findall(r'\b[a-zA-Z]+\b', all_text.lower())
    words = [w for w in words if w not in stop_words]
    ngrams = zip(*[words[i:] for i in range(n)])
    return Counter([" ".join(ngram) for ngram in ngrams]).most_common(top_n)

def get_tfidf(subreddit_texts, top_n=10):
    vectorizer = TfidfVectorizer(stop_words='english', max_features=top_n)
    tfidf_matrix = vectorizer.fit_transform(subreddit_texts)
    feature_names = vectorizer.get_feature_names_out()
    scores = tfidf_matrix.sum(axis=0).A1
    return sorted(zip(feature_names, scores), key=lambda x: x[1], reverse=True)

def plot_word_frequencies(subreddit_texts, subreddit_name):
    word_counts = get_common_words(subreddit_texts)
    if not word_counts:
        print(f"No significant words found for subreddit {subreddit_name}")
        return

    words, counts = zip(*word_counts)
    plt.figure(figsize=(10, 5))
    plt.bar(words, counts, color='skyblue', edgecolor='black')
    plt.xlabel("Words")
    plt.ylabel("Frequency")
    plt.title(f"Most Common Words in r/{subreddit_name}")
    plt.xticks(rotation=45)
    plt.show()

def build_word_network(subreddit_texts, subreddit_name):
    all_text = " ".join(subreddit_texts)
    words = re.findall(r'\b[a-zA-Z]+\b', all_text)
    words = [word.lower() for word in words if word.lower() not in ENGLISH_STOP_WORDS]

    G = nx.Graph()
    for i in range(len(words) - 1):
        G.add_edge(words[i], words[i+1])

    plt.figure(figsize=(12, 8))
    pos = nx.spring_layout(G, seed=42)
    nx.draw(G, pos, node_size=20, edge_color='gray', alpha=0.7, with_labels=False)
    plt.title(f"Word Network Graph for r/{subreddit_name}")
    plt.show()

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter

def calculate_word_statistics(word_list, total_words_in_emotion, total_texts_in_emotion, total_texts):
    """
    Calculates comprehensive word statistics including frequency, TF, log-TF, IDF, TF-IDF, and BM25.

    :param word_list: List of words that appeared with a specific emotion.
    :param total_words_in_emotion: Total number of words across all texts for the emotion.
    :param total_texts_in_emotion: Total number of texts associated with the specific emotion.
    :param total_texts: Total number of texts in the entire dataset.
    :return: A DataFrame with the statistics for each word.
    """
    # Count the frequency of each word
    word_counts = Counter(word_list)

    # Create a DataFrame with the word and its frequency
    df = pd.DataFrame(word_counts.items(), columns=['Word', 'Frequency'])

    # Calculate Term Frequency (TF)
    df['TF'] = df['Frequency'] / total_words_in_emotion

    # Calculate log-transformed TF (log-TF)
    df['Log-TF'] = np.log1p(df['TF'])  # log1p to avoid log(0)

    # Calculate Inverse Document Frequency (IDF)
    df['IDF'] = np.log((total_texts + 1) / (total_texts_in_emotion + 1))  # Add 1 to avoid division by zero

    # Calculate TF-IDF
    df['TF-IDF'] = df['TF'] * df['IDF']

    # Calculate BM25 components
    k = 1.5  # Tunable parameter
    b = 0.75  # Tunable parameter
    avg_doc_length = total_words_in_emotion / total_texts_in_emotion
    df['BM25'] = (df['Frequency'] * (k + 1)) / (df['Frequency'] + k * (1 - b + b * (total_words_in_emotion / avg_doc_length)))

    # Calculate a custom importance metric (example: normalized frequency squared)
    df['Custom-Metric'] = (df['Frequency'] / total_words_in_emotion) ** 2

    return df.sort_values(by='Frequency', ascending=False)







In [ ]:
list_word_emotoin = []
for i in emotion_columns:
  dict_word = emotion_word_dict[i]
  total_words = len(dict_word)
  total_texts_in_i = combined_df[combined_df[i] == 1].shape[0]
  total_texts = combined_df.shape[0]
  word_statistics = calculate_word_statistics(dict_word, total_words,total_texts_in_i,total_texts)

  list_word_emotoin.append(word_statistics)

In [ ]:
for i in range(len(list_word_emotoin)):
  print(emotion_columns[i])
  print(list_word_emotoin[i].head(10))

admiration
        Word  Frequency        TF    Log-TF       IDF    TF-IDF      BM25  \
30      name       3202  0.028757  0.028352  2.511981  0.072238  0.356177   
1       good       2372  0.021303  0.021079  2.511981  0.053513  0.273969   
88     great       1984  0.017818  0.017662  2.511981  0.044760  0.233338   
22      like       1446  0.012987  0.012903  2.511981  0.032622  0.174480   
47      love       1107  0.009942  0.009893  2.511981  0.024974  0.135797   
109  awesome        951  0.008541  0.008505  2.511981  0.021455  0.117560   
6        one        945  0.008487  0.008451  2.511981  0.021320  0.116853   
151  amazing        870  0.007814  0.007783  2.511981  0.019627  0.107979   
63    really        862  0.007742  0.007712  2.511981  0.019447  0.107029   
43      best        851  0.007643  0.007614  2.511981  0.019199  0.105721   

     Custom-Metric  
30        0.000827  
1         0.000454  
88        0.000317  
22        0.000169  
47        0.000099  
109       0.000

In [ ]:
words_for_joy = emotion_word_dict['joy']
total_words_in_joy = len(words_for_joy)
total_texts_in_joy = combined_df[combined_df['joy'] == 1].shape[0]
total_texts = combined_df.shape[0]
# Compute the statistics
word_statistics = calculate_word_statistics(words_for_joy, total_words_in_joy,total_texts_in_joy,total_texts)
print(word_statistics)

              Word  Frequency        TF    Log-TF       IDF    TF-IDF  \
0            happy       1185  0.022226  0.021983  3.275489  0.072801   
1             name       1167  0.021888  0.021652  3.275489  0.071695   
35            glad        885  0.016599  0.016463  3.275489  0.054370   
27            like        675  0.012660  0.012581  3.275489  0.041469   
15            love        607  0.011385  0.011321  3.275489  0.037291   
...            ...        ...       ...       ...       ...       ...   
50         squeals          1  0.000019  0.000019  3.275489  0.000061   
4719  unattractive          1  0.000019  0.000019  3.275489  0.000061   
4720    appearance          1  0.000019  0.000019  3.275489  0.000061   
4722    deservedly          1  0.000019  0.000019  3.275489  0.000061   
7711       hostile          1  0.000019  0.000019  3.275489  0.000061   

          BM25  Custom-Metric  
0     0.291405   4.939939e-04  
1     0.287488   4.791005e-04  
35    0.224249   2.755316e-

In [ ]:
def find_word_overlap(emotion_word_dict):
    """
    Finds overlapping words between different emotion categories and calculates their occurrences
    across the emotions.

    :param emotion_word_dict: Dictionary where keys are emotions and values are lists of words.
    :return: A DataFrame showing words, the emotions they appear in, and their counts in each emotion.
    """
    from collections import defaultdict

    # Create a dictionary to track words across emotions
    word_emotion_count = defaultdict(lambda: defaultdict(int))

    # Count occurrences of each word per emotion
    for emotion, words in emotion_word_dict.items():
        for word in words:
            word_emotion_count[word][emotion] += 1

    # Convert to DataFrame
    rows = []
    for word, emotions in word_emotion_count.items():
        row = {'Word': word}
        row.update(emotions)  # Add counts for each emotion
        rows.append(row)

    df = pd.DataFrame(rows)
    df = df.fillna(0).astype({'Word': 'str'})  # Replace NaN with 0
    return df.sort_values(by='Word')

In [ ]:
overlap_df = find_word_overlap(emotion_word_dict)


In [ ]:
def calculate_metrics(df, emotion_columns):
    """
    Calculates multiple metrics for a DataFrame with words and their occurrences across specific emotions.

    :param df: DataFrame where the first column is "Word" and subsequent columns are emotion counts.
    :param emotion_columns: List of columns corresponding to specific emotions.
    :return: DataFrame with additional metrics.
    """
    # Copy the DataFrame to avoid overwriting the original
    df = df.copy()

    # Convert emotion columns to numeric, replacing non-numeric values with 0
    for col in emotion_columns:
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)

    # Add EmotionCount (number of emotions with non-zero values for the word)
    df['EmotionCount'] = (df[emotion_columns] > 0).sum(axis=1)

    # Add EmotionsWithWord (list of emotions where the word appears)
    df['EmotionsWithWord'] = df[emotion_columns].apply(
        lambda x: ', '.join([col for col in emotion_columns if x[col] > 0]), axis=1
    )

    # 1. Normalization by emotion (relative weight of each word in the emotion)
    for emotion in emotion_columns:
        total = df[emotion].sum()
        if total > 0:
            df[f'Normalized_{emotion}'] = df[emotion] / total
        else:
            df[f'Normalized_{emotion}'] = 0

    # 2. Overlap score (number of emotions the word appears in)
    df['Overlap'] = df[emotion_columns].astype(bool).sum(axis=1)

    # 3. TF-IDF for each emotion
    N = len(emotion_columns)  # Number of emotions
    df['DF'] = df[emotion_columns].gt(0).sum(axis=1)  # Document Frequency
    for emotion in emotion_columns:
        total = df[emotion].sum()
        if total > 0:
            df[f'TF-IDF_{emotion}'] = (df[emotion] / total) * np.log(N / df['DF'])
        else:
            df[f'TF-IDF_{emotion}'] = 0

    # 4. Unique word indicator (appears in only one emotion)
    df['IsUnique'] = (df['Overlap'] == 1).astype(int)

    return df


    return df
overlap_df_stas = calculate_metrics(overlap_df,emotion_columns)
# overlap_df_stas = overlap_df_stas.drop(columns=emotion_columns)

In [ ]:
overlap_df_stas.describe()

,sadness,neutral,love,gratitude,disapproval,amusement,disappointment,disgust,admiration,realization,...,TF-IDF_nervousness,TF-IDF_optimism,TF-IDF_pride,TF-IDF_realization,TF-IDF_relief,TF-IDF_remorse,TF-IDF_sadness,TF-IDF_surprise,TF-IDF_neutral,IsUnique
count,29671.000000,29671.000000,29671.000000,29671.000000,29671.000000,29671.000000,29671.000000,29671.000000,29671.000000,29671.000000,...,29671.000000,29671.000000,29671.000000,29671.000000,29671.000000,29671.000000,29671.000000,29671.000000,29671.000000,29671.000000
mean,1.578039,12.123285,1.811803,2.372788,2.821004,2.118803,2.116174,1.269792,3.752654,2.235179,...,0.000008,0.000008,0.000008,0.000009,0.000007,0.000007,0.000008,0.000009,0.000012,0.073405
std,13.509546,100.725412,34.478431,39.623930,25.877863,25.770088,16.472488,10.034399,36.242097,16.715647,...,0.000033,0.000017,0.000039,0.000016,0.000040,0.000034,0.000021,0.000027,0.000011,0.260805
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000003,0.000000
50%,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000010,0.000000
75%,0.000000,5.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,2.000000,1.000000,...,0.000000,0.000008,0.000000,0.000014,0.000000,0.000000,0.000000,0.000000,0.000017,0.000000
max,994.000000,12194.000000,5132.000000,4701.000000,2679.000000,3271.000000,1475.000000,876.000000,3202.000000,1337.000000,...,0.001511,0.000288,0.000908,0.000329,0.001145,0.000769,0.000559,0.000810,0.000417,1.000000


#word2vec

In [ ]:
from gensim.models import Word2Vec
from scipy.spatial.distance import cosine

def train_word2vec_by_emotion(emotion_dict, vector_size=100, window=5, min_count=1, epochs=10):
    """
    Train Word2Vec models for each emotion and a combined model for all words.

    :param emotion_dict: Dictionary where keys are emotion labels and values are lists of words for each emotion.
    :param vector_size: Size of the embedding vectors.
    :param window: Maximum distance between the current and predicted word within a sentence.
    :param min_count: Ignores all words with total frequency lower than this.
    :param epochs: Number of training iterations for the Word2Vec model.
    :return: Dictionary of Word2Vec models for each emotion, including a combined 'AllWords' model.
    """
    emotion_models = {}

    # Train Word2Vec model for each emotion
    for emotion, words in emotion_dict.items():
        sentences = [[word] for word in words]  # Treat each word as its own "sentence"
        model = Word2Vec(sentences=sentences, vector_size=vector_size, window=window, min_count=min_count, epochs=epochs)
        emotion_models[emotion] = model

    # Combine all words into a single list for 'AllWords'
    all_words = []
    for words in emotion_dict.values():
        all_words.extend(words)
    all_sentences = [[word] for word in all_words]

    # Train the combined 'AllWords' model
    all_words_model = Word2Vec(sentences=all_sentences, vector_size=vector_size, window=window, min_count=min_count, epochs=epochs)
    emotion_models['AllWords'] = all_words_model

    return emotion_models

def calculate_emotion_distances(emotion_models):
    """
    Calculate pairwise vector distances between emotions based on Word2Vec embeddings.

    :param emotion_models: Dictionary of Word2Vec models trained for each emotion.
    :return: Dictionary with distances between each pair of emotions.
    """
    emotion_vectors = {}
    # Calculate the mean vector for each emotion
    for emotion, model in emotion_models.items():
        words = list(model.wv.index_to_key)
        vectors = [model.wv[word] for word in words]
        emotion_vectors[emotion] = sum(vectors) / len(vectors)  # Mean vector for the emotion

    # Compute pairwise distances
    distances = {}
    emotions = list(emotion_vectors.keys())
    for i, emotion1 in enumerate(emotions):
        for j, emotion2 in enumerate(emotions):
            if i < j:  # Avoid duplicate calculations
                dist = cosine(emotion_vectors[emotion1], emotion_vectors[emotion2])
                distances[(emotion1, emotion2)] = dist
    return distances
emotion_models = train_word2vec_by_emotion(emotion_word_dict, vector_size=50, epochs=20)


In [ ]:
dist_emotion = calculate_emotion_distances(emotion_models)

In [ ]:
distance_df = pd.DataFrame(
    [(emotion1, emotion2, distance) for (emotion1, emotion2), distance in dist_emotion.items()],
    columns=["Emotion 1", "Emotion 2", "Distance"]
)
distance_df

,Emotion 1,Emotion 2,Distance
0,sadness,neutral,0.578517
1,sadness,love,0.055607
2,sadness,gratitude,0.059697
3,sadness,disapproval,0.214072
4,sadness,amusement,0.030926
...,...,...,...
401,desire,fear,0.061498
402,desire,AllWords,0.670138
403,relief,fear,0.242912
404,relief,AllWords,0.798786


#attinsh is all wht you need

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class EmotionVectorModel(nn.Module):
    def __init__(self, embed_dim, num_heads, vocab_size, num_emotions):
        super(EmotionVectorModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.attention = nn.MultiheadAttention(embed_dim=embed_dim, num_heads=num_heads, batch_first=True)
        self.emotion_embedding = nn.Embedding(num_emotions, embed_dim)

    def forward(self, input_ids, attention_mask, emotion_id):
        # Embedding ראשי של מילים
        embeddings = self.embedding(input_ids)

        # וקטור רגשי נוסף
        emotion_vector = self.emotion_embedding(emotion_id).unsqueeze(1)
        embeddings = embeddings + emotion_vector

        # Mask לטיפול במילים ריקות (PAD)
        key_padding_mask = ~attention_mask.to(torch.bool)

        # Self-Attention
        attn_output, _ = self.attention(
            embeddings, embeddings, embeddings, key_padding_mask=key_padding_mask
        )

        return attn_output  # מחזיר ייצוגים וקטוריים לכל מילה


In [ ]:
def create_emotion_dict(dataframe, tokenizer, emotion_columns):

    emotion_dict = {emotion: [] for emotion in emotion_columns}

    for _, row in dataframe.iterrows():
        text = " ".join(row['preprocessed_text'])

        tokenized = tokenizer(
            text,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
            max_length=128
        )

        for emotion in emotion_columns:
            if row[emotion] == 1:
                emotion_dict[emotion].append({
                    'input_ids': tokenized['input_ids'].squeeze(0),
                    'attention_mask': tokenized['attention_mask'].squeeze(0),
                    'label': torch.tensor(1.0, dtype=torch.float)
                })
    return emotion_dict


In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

emotion_dict = create_emotion_dict(combined_df, tokenizer, emotion_columns)


In [ ]:
def generate_emotion_vectors(emotion_dict, model):

    emotion_vectors = {}

    for emotion, examples in emotion_dict.items():
        emotion_vectors[emotion] = []
        for example in examples:
            input_ids = example['input_ids'].unsqueeze(0)
            attention_mask = example['attention_mask'].unsqueeze(0)
            emotion_id = torch.tensor([emotion_columns.index(emotion)]).long()

            with torch.no_grad():
                vectors = model(input_ids, attention_mask, emotion_id)

            emotion_vectors[emotion].append(vectors.squeeze(0).numpy())

    return emotion_vectors


In [ ]:
# הגדרת המודל
embed_dim = 128
num_heads = 8
vocab_size = 10000
num_emotions = len(emotion_columns)
model = EmotionVectorModel(embed_dim, num_heads, vocab_size, num_emotions)

# יצירת ווקטורים
emotion_vectors = generate_emotion_vectors(emotion_dict, model)

# הצגת התוצאות
for emotion, vectors in emotion_vectors.items():
    print(f"Emotion: {emotion}, Number of examples: {len(vectors)}")
    print(f"First example vector shape: {vectors[0].shape}")


AttributeError: 'Index' object has no attribute 'index'

In [ ]:
from scipy.spatial.distance import cosine, euclidean

def calculate_word_distances(emotion_vectors, method="cosine"):
    """
    מחשב מרחקים בין ווקטורים של מילים עבור כל רגש.

    Args:
        emotion_vectors: מילון של רגשות עם ווקטורים לכל מילה.
        method: סוג המדד לחישוב המרחק ('cosine' או 'euclidean').

    Returns:
        מילון של רגשות עם רשימות מרחקים.
    """
    distances = {}

    for emotion, vectors in emotion_vectors.items():
        distances[emotion] = []
        for vector_set in vectors:
            for i in range(len(vector_set)):
                for j in range(i + 1, len(vector_set)):
                    if method == "cosine":
                        dist = cosine(vector_set[i], vector_set[j])
                    elif method == "euclidean":
                        dist = euclidean(vector_set[i], vector_set[j])
                    distances[emotion].append(dist)
    return distances

# חישוב המרחקים
distances = calculate_word_distances(emotion_vectors, method="cosine")
print(distances)


In [ ]:
import torch
import torch.nn as nn

class EmotionVectorModel(nn.Module):
    def __init__(self, embed_dim, num_heads, vocab_size, num_emotions):
        super(EmotionVectorModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.attention = nn.MultiheadAttention(embed_dim=embed_dim, num_heads=num_heads, batch_first=True)
        self.emotion_embedding = nn.Embedding(num_emotions, embed_dim)

    def forward(self, input_ids, attention_mask, emotion_id):
        embeddings = self.embedding(input_ids)

        emotion_vector = self.emotion_embedding(emotion_id).unsqueeze(1)

        embeddings = embeddings + emotion_vector

        key_padding_mask = ~attention_mask.to(torch.bool)

        attn_output, _ = self.attention(
            embeddings, embeddings, embeddings, key_padding_mask=key_padding_mask
        )

        return attn_output


In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(text):
    return tokenizer(
        text,
        max_length=128,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )


In [ ]:
emotion_map = {emotion: idx for idx, emotion in enumerate(emotion_columns)}


In [ ]:
embed_dim = 64
num_heads = 4
vocab_size = tokenizer.vocab_size
num_emotions = len(emotion_columns)

model = EmotionVectorModel(embed_dim, num_heads, vocab_size, num_emotions)


text = combined_df['preprocessed_text']
tokenized = tokenize_function(text)
input_ids = tokenized['input_ids']
attention_mask = tokenized['attention_mask']
emotion_id = torch.tensor([emotion_map['joy']])

with torch.no_grad():
    vector_output = model(input_ids, attention_mask, emotion_id)

print("Vector output shape:", vector_output.shape)


ValueError: text input must be of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

tensor([[[-0.6110, -0.0597, -0.0600,  ..., -0.1180, -0.2386,  0.8235],
         [-0.6320,  0.1860,  0.5712,  ..., -0.2277, -0.2162,  0.7453],
         [-0.7074,  0.0901,  0.2041,  ..., -0.2522, -0.1497,  0.8194],
         ...,
         [-0.6724,  0.0412,  0.1083,  ..., -0.2751, -0.2504,  0.8317],
         [-0.6724,  0.0412,  0.1083,  ..., -0.2751, -0.2504,  0.8317],
         [-0.6724,  0.0412,  0.1083,  ..., -0.2751, -0.2504,  0.8317]]])

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(text):
    return tokenizer(
        text,
        max_length=128,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
    )

combined_df['tokenized'] = combined_df['preprocessed_text'].apply(
    lambda x: tokenize_function(" ".join(x))
)


In [ ]:
emotion_tokenizer_dict = build_emotion_word_dict(combined_df, text_column='tokenized', emotion_columns=emotion_columns)


In [ ]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(combined_df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(test_df, test_size=0.5, random_state=42)


In [ ]:
class GoEmotionsSingleLabelDataset(Dataset):
    def __init__(self, dataframe, tokenizer, target_emotion, max_length=128):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.target_emotion = target_emotion
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        text = " ".join(row['preprocessed_text'])

        # Convert target emotion to tensor (Binary classification: 1 or 0)
        label = torch.tensor(row[self.target_emotion], dtype=torch.float)

        encoded = self.tokenizer(
            text,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        return {
            'input_ids': encoded['input_ids'].squeeze(0),
            'attention_mask': encoded['attention_mask'].squeeze(0),
            'label': label
        }


In [ ]:
joy_dataset = GoEmotionsSingleLabelDataset(train_df, tokenizer, target_emotion="joy")
sample = joy_dataset[0]

for i in range(5):
    sample = joy_dataset[i]
    print(f"Sample {i}:")
    print("Text IDs:", sample['input_ids'])
    print("Label:", sample['label'])
    print("Attention Mask:", sample['attention_mask'])
    print("---")

In [ ]:
class TransformerEmotionClassifier(nn.Module):
    def __init__(self, embed_dim, num_heads, vocab_size):
        super(TransformerEmotionClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.attention = nn.MultiheadAttention(embed_dim=embed_dim, num_heads=num_heads, batch_first=True)
        self.fc = nn.Linear(embed_dim, 1)

    def forward(self, input_ids, attention_mask):
        embeddings = self.embedding(input_ids)

        # Ensure attention_mask is bool and in correct shape
        attention_mask = attention_mask.to(torch.bool).T

        attn_output, _ = self.attention(
            embeddings, embeddings, embeddings, key_padding_mask=~attention_mask
        )

        pooled_output = attn_output.mean(dim=1)
        return self.fc(pooled_output)


In [ ]:
embed_dim = 64
num_heads = 4
vocab_size = 30522


emotion_models = {emotion: EmotionAttentionModel(embed_dim, num_heads, vocab_size) for emotion in emotion_columns}


In [ ]:
def train_emotion_model(model, dataloader, optimizer, criterion, num_epochs=3, device='cpu'):
    model.to(device)
    model.train()

    for epoch in range(num_epochs):
        total_loss = 0
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)  # ודא שזה נכון
            labels = batch['labels'].to(device)

            # Ensure attention_mask is bool
            attention_mask = attention_mask.to(torch.bool)

            optimizer.zero_grad()
            logits = model(input_ids, attention_mask)
            loss = criterion(logits.view(-1), labels.view(-1))
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / len(dataloader):.4f}")


In [ ]:
from torch.utils.data import DataLoader

# יצירת DataLoader עבור רגשות
batch_size = 16
for emotion in emotion_columns:
    # DataLoader עבור הרגש
    emotion_dataset = GoEmotionsDataset(train_df, tokenizer, [emotion])
    emotion_dataloader = DataLoader(emotion_dataset, batch_size=batch_size, shuffle=True)
    print(emotion_dataset)
    # הגדרת המודל, קריטריון, ואופטימיזציה
    model = emotion_models[emotion]
    optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
    criterion = nn.BCEWithLogitsLoss()

    # אימון
    print(f"Training model for emotion: {emotion}")
    train_emotion_model(model, emotion_dataloader, optimizer, criterion)


Training model for emotion: admiration


KeyboardInterrupt: 

In [ ]:
def predict_emotion(model, dataloader, device='cuda'):
    model = model.to(device)
    model.eval()
    predictions = []
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            logits = model(input_ids, attention_mask)
            preds = torch.sigmoid(logits).cpu().numpy()
            predictions.extend(preds)
    return predictions


In [ ]:
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

for emotion in emotion_columns:
    model = emotion_models[emotion]
    print(f"Predicting for emotion: {emotion}")
    predictions = predict_emotion(model, test_dataloader)
    # חישוב המטריקות (F1, Precision, Recall וכו')


#Vocabulary

In [ ]:
import numpy as np
from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity
import torch

class Vocabulary:
    def __init__(self, default_indexes=None):
        if default_indexes is None:
            default_indexes = {}
        self.default_indexes = default_indexes
        self.init()

    def init(self):
        self.index_to_word = {**self.default_indexes}
        self.word_to_index = {word: idx for idx, word in self.index_to_word.items()}
        self.word_counts = Counter()
        self.num_words = len(self.default_indexes)
        for word in self.index_to_word:
            self.word_counts[word] = 1
        self.embedding_matrix = None
        self.vector_size = 300

    def index_words(self, word_list):
        for word in word_list:
            self.index_word(word)

    def __len__(self):
        return len(self.word_to_index)

    def index_word(self, word, cnt=None):
        if word not in self.word_to_index:
            idx = len(self.index_to_word)
            self.index_to_word[idx] = word
            self.word_to_index[word] = idx
            self.word_counts[word] = cnt if cnt is not None else 1
            self.num_words += 1
        else:
            self.word_counts[word] += cnt if cnt is not None else 1

    def get_words(self, indices):
        return [self.index_to_word.get(i, None) for i in indices]

    def build_embedding_matrix(self, word2vec_model, embed_dim=300, init='random'):
        if init == 'zeros':
            self.embedding_matrix = np.zeros((self.num_words, embed_dim))
        elif init == 'random':
            self.embedding_matrix = np.random.rand(self.num_words, embed_dim)
        else:
            raise ValueError("Unknown initialization method")

        for idx, word in self.index_to_word.items():
            if word in word2vec_model:
                self.embedding_matrix[idx] = word2vec_model[word]
            else:
                if init == 'random':
                    self.embedding_matrix[idx] = np.random.rand(embed_dim)
                else:
                    self.embedding_matrix[idx] = np.zeros(embed_dim)

        return self.embedding_matrix

    def get_word_vector(self, word):
        word_idx = self.get_word_index(word)
        if word_idx is not None:
            return self.embedding_matrix[word_idx]
        print(f"Word '{word}' not found in vocabulary!")
        return None

    def get_word_index(self, word):
        return self.word_to_index.get(word, None)

    def get_word_from_index(self, index):
        return self.index_to_word.get(index, None)

    def apply_attention(self, sequence, output, temperature=1.0):
        attention_weights = torch.softmax(output / temperature, dim=-1)
        weighted_sequence = torch.sum(attention_weights.unsqueeze(-1) * sequence, dim=1)
        return weighted_sequence

    def get_most_similar_words(self, word_vector, top_n=20):
        similarities = cosine_similarity([word_vector], self.embedding_matrix)
        similar_indices = similarities[0].argsort()[-top_n:][::-1]
        similar_words = [(self.index_to_word[idx], similarities[0][idx]) for idx in similar_indices]
        return similar_words

    def preprocess_goemotions_data(self, dataframe, text_column):
        dataframe[text_column] = dataframe[text_column].str.lower()
        dataframe[text_column] = dataframe[text_column].str.replace(r'[^a-z\s]', '', regex=True)
        dataframe[text_column] = dataframe[text_column].str.split()
        for text in dataframe[text_column]:
            self.index_words(text)

    def get_combined_features(self, word):
        word_vector = self.get_word_vector(word)
        if word_vector is None:
            word_vector = np.zeros(self.vector_size)
        return word_vector
